# Imports

In [1]:
%pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.7 MB/s eta 0:00:00


In [2]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

from scipy.stats import randint, uniform
from tabulate import tabulate
import numpy as np
np.random.seed(42)
import pandas as pd

import tensorflow as tf
tf.random.set_seed(42)
import keras
import keras_tuner as kt
import joblib

# Dataset

## Features

1. destination[No Urgent Place, Home, Work]
2. passenger[Alone, Friend(s), Kid(s), Partner]
3. weather[Sunny, Rainy, Snowy]
4. temperatures[55, 80, 30]
5. time[2PM, 10AM, 6PM, 7AM, 10PM]
6. coupon[Restaurant(<20), Coffee House, Carry out & Take away, Bar, Restaurant(20-50)]
7. expiration[1d, 2h]
8. gender[Female, Male]
9. age[21, 46, 26, 31, 41, 50plus, 36, below21]
10. maritalStatus[Unmarried partner, Single, Married partner, Divorced, Widowed]
11. has_Children[0, 1]
12. education[ Some college - no degree, Bachelors degree, Associates degree, High School Graduate, Graduate degree (Masters or Doctorate), Some High School]
13. occupation[Unemployed, Architecture & Engineering, Student,
Education&Training&Library, Healthcare Support,
Healthcare Practitioners & Technical, Sales & Related, Management,
Arts Design Entertainment Sports & Media, Computer & Mathematical,
Life Physical Social Science, Personal Care & Service,
Community & Social Services, Office & Administrative Support,
Construction & Extraction, Legal, Retired,
Installation Maintenance & Repair, Transportation & Material Moving,
Business & Financial, Protective Service,
Food Preparation & Serving Related, Production Occupations,
Building & Grounds Cleaning & Maintenance, Farming Fishing & Forestry]
14. income[$37500 - $49999, $62500 - $74999, $12500 - $24999, $75000 - $87499,
$50000 - $62499, $25000 - $37499, $100000 or More, $87500 - $99999, Less than $12500]
15. car[]
16. Bar[ever, less1, 1-3, gt8,  nan4~8] : howm many times do you go to a bar every month
17. Coffee House[never, less1, 4-8, 1-3, gt8,  nan]: How many times do you go to a coffehouse every month
18. Carry Away[n4~8, 1-3, gt8, less1, never]:  how many times do you get take-away food every month
19. RestaurantLessThan20[ 4-8, 1-3, less1, gt8, never]: how many times do you go to a restaurant with an average expense per person of less than 20 every month
20. Restaurant20To50[1-3, less1, never, gt8, 4-8,  nan]: how many times do you go to a restaurant with average expense per person of 20 - $50 every month
21. toCoupon_GE5min[0, 1]: driving distance to the restaurant/bar for using the coupon is greater than 5 minutes
22. toCoupon_GEQ15min[0,1]: driving distance to the restaurant/bar for using the coupon is greater than 15 minutes
23. toCoupon_GEQ25min[0, 1]: driving distance to the restaurant/bar for using the coupon is greater than 25 minutes
24. direction_same[0,1]: whether the restaurant/bar is in the same direction as your current destination
25. direction_op[0, 1]: whether the restaurant/bar is in the same direction as your current destination
26. Y[0, 1]: whether the coupon is accepted

## Looking at the Dataset

In [3]:
dataset = pd.read_csv('https://raw.githubusercontent.com/NikitasThermos/In-Vehicle-Recommendation-System/main/Dataset/in-vehicle-coupon-recommendation.csv')

In [ ]:
dataset.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


In [ ]:
dataset.shape

(12684, 26)

In [ ]:
print(f'Number of coupons: {dataset.shape[0]}')
print(f'Number of declined coupons: {np.sum(dataset["Y"]==0)}')
print(f'Number of accepted coupons: {np.sum(dataset["Y"]==1)}')

Number of coupons: 12684
Number of negative class: 5474
Number of positive class: 7210


In [4]:
training_df, labels = dataset.drop('Y', axis=1), dataset['Y']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(training_df, labels, test_size=0.2, random_state=42)

In [ ]:
missing_percentage = X_train.isnull().mean() * 100
for col, per in zip(missing_percentage.index, missing_percentage):
  print(f'{col}, {per:.2f}%')

destination, 0.00%
passanger, 0.00%
weather, 0.00%
temperature, 0.00%
time, 0.00%
coupon, 0.00%
expiration, 0.00%
gender, 0.00%
age, 0.00%
maritalStatus, 0.00%
has_children, 0.00%
education, 0.00%
occupation, 0.00%
income, 0.00%
car, 99.23%
Bar, 0.83%
CoffeeHouse, 1.77%
CarryAway, 1.17%
RestaurantLessThan20, 1.01%
Restaurant20To50, 1.48%
toCoupon_GEQ5min, 0.00%
toCoupon_GEQ15min, 0.00%
toCoupon_GEQ25min, 0.00%
direction_same, 0.00%
direction_opp, 0.00%


# Preprocessing

In [6]:


def add_columns(X):
       X['distance_to_coupon'] = X['toCoupon_GEQ5min'] + X['toCoupon_GEQ15min'] + X['toCoupon_GEQ25min']
       X = X.drop(columns=['toCoupon_GEQ5min', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min'])
       return X


category_order = {'temperature': [30, 55, 80],
                  'time': ['7AM', '10AM', '2PM', '6PM', '10PM'],
                  'age': ['below21', '21', '26', '31', '36', '41', '46', '50plus'],
                  'education': ['Some High School', 'High School Graduate', 'Some college - no degree', 'Associates degree', 'Bachelors degree', 'Graduate degree (Masters or Doctorate)'],
                  'income': ['Less than $12500', '$12500 - $24999', '$25000 - $37499','$37500 - $49999', '$50000 - $62499', '$62500 - $74999', '$75000 - $87499', '$87500 - $99999', '$100000 or More'],
                  'Bar': ['never', 'less1', '1~3', '4~8', 'gt8'],
                  'CoffeeHouse': ['never', 'less1', '1~3', '4~8', 'gt8'],
                  'CarryAway': ['never', 'less1', '1~3', '4~8', 'gt8'],
                  'RestaurantLessThan20': ['never', 'less1', '1~3', '4~8', 'gt8'],
                  'Restaurant20To50': ['never', 'less1', '1~3', '4~8', 'gt8']}

order_category_names = ['temperature', 'time', 'age',  'education', 'income', 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20',  'Restaurant20To50']
non_order_category_names = ['destination', 'passanger', 'weather', 'coupon', 'expiration', 'gender', 'maritalStatus', 'occupation']
drop = ['direction_opp', 'car']

order_encoder = OrdinalEncoder(categories=[category_order[feature] for feature in order_category_names],
                               handle_unknown='use_encoded_value',
                               unknown_value=-1)
add_columns_transformer = FunctionTransformer(add_columns, validate=False)


preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', order_encoder, order_category_names),
        ('encode', OrdinalEncoder(), non_order_category_names),
        ('add_columns', add_columns_transformer, ['toCoupon_GEQ5min', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min']),
        ('drop', 'drop', drop)],
    remainder='passthrough',
    verbose_feature_names_out=False).set_output(transform='pandas')

## Perfomance Measures

In [7]:
results = []
def log_results(model, y_true, y_pred):
  conf_matrix = confusion_matrix(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)
  roc_auc = roc_auc_score(y_true, y_pred)
  print(f'confusion matrix for {model}:\n {conf_matrix}')
  print(f'Precision Score: {precision:.2f}')
  print(f'Recall Score: {recall:.2f}')
  print(f'F1 Score: {f1:.2f}')
  print(f'ROC_AUC Score: {roc_auc:.2f}')
  results.append([model, precision, recall, f1, roc_auc])

# SGDClassifier

In [10]:
full_pipeline = make_pipeline(preprocessor,
                              SGDClassifier(loss='log_loss',
                                            learning_rate='adaptive', random_state=42))

param_grid = {
    'sgdclassifier__alpha': uniform(loc=0.0001, scale=3),
    'sgdclassifier__eta0': uniform(loc=0.0001, scale=10),
}

random_search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_grid,
    n_iter=10,
    scoring='roc_auc',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)
random_search.fit(X_train, y_train)
random_search.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'sgdclassifier__alpha': 0.06185348288740734,
 'sgdclassifier__eta0': 9.699198521619943}

In [11]:
y_test_pred = random_search.predict(X_test)
log_results('SGDClassifier', y_test, y_test_pred)

confusion matrix for SGDClassifier:
 [[ 346  782]
 [ 223 1186]]
Precision Score: 0.60
Recall Score: 0.84
F1 Score: 0.70
ROC_AUC Score: 0.57


In [12]:
joblib.dump(random_search, 'logloss.pki')

['logloss.pki']

# Decision Tree

In [13]:
full_pipeline = make_pipeline(preprocessor,
                              DecisionTreeClassifier(random_state=42))

param_grid = {
    'decisiontreeclassifier__criterion': ['gini', 'entropy'],
    'decisiontreeclassifier__min_samples_split': randint(low=2, high=10),
    'decisiontreeclassifier__max_depth': randint(low=1, high=10),
}

random_search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_grid,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42,
)

random_search.fit(X_train, y_train)
random_search.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'decisiontreeclassifier__criterion': 'gini',
 'decisiontreeclassifier__max_depth': 8,
 'decisiontreeclassifier__min_samples_split': 6}

In [14]:
y_test_pred = random_search.predict(X_test)
log_results('Decision Tree', y_test, y_test_pred)

confusion matrix for Decision Tree:
 [[ 654  474]
 [ 301 1108]]
Precision Score: 0.70
Recall Score: 0.79
F1 Score: 0.74
ROC_AUC Score: 0.68


In [ ]:
joblib.dump(random_search, 'decisiontree.pki')

['decisiontree.pki']

# Random Forest

In [15]:
full_pipeline = make_pipeline(preprocessor,
                              RandomForestClassifier(random_state=42))

param_grid = {
    'randomforestclassifier__n_estimators': randint(low=5, high=20),
    'randomforestclassifier__max_depth': randint(low=1, high=15),
    'randomforestclassifier__criterion': ['gini', 'entropy'],
}

random_search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_grid,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42,
)

random_search.fit(X_train, y_train)
random_search.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'randomforestclassifier__criterion': 'gini',
 'randomforestclassifier__max_depth': 11,
 'randomforestclassifier__n_estimators': 12}

In [ ]:
y_test_pred = random_search.predict(X_test)
log_results('Random Forest', y_test, y_test_pred)

confusion matrix for Random Forest:
 [[ 667  461]
 [ 239 1170]]
Precision Score: 0.72
Recall Score: 0.83
F1 Score: 0.77
ROC_AUC Score: 0.71


In [ ]:
joblib.dump(random_search, 'rf.pki')

['rf.pki']

# SVM

In [16]:
full_pipeline = make_pipeline(preprocessor,
                              SVC(kernel='rbf', gamma='scale', degree=2, random_state=42))

param_grid = {
    'svc__C': uniform(loc=0.001, scale=10),
}

random_search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_grid,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42,
)
random_search.fit(X_train, y_train)
random_search.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


KeyboardInterrupt: 

In [ ]:
y_test_pred = random_search.predict(X_test)
log_results('SVM', y_test, y_test_pred)

confusion matrix for SVM:
 [[ 512  616]
 [ 254 1155]]
Precision Score: 0.65
Recall Score: 0.82
F1 Score: 0.73
ROC_AUC Score: 0.64


In [ ]:
joblib.dump(random_search, 'svm.pki')

['svm.pki']

# DNN

In [8]:
X_train_nn, X_val_nn, y_train_nn, y_val_nn = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [9]:
X_train_nn = pd.DataFrame(X_train_nn, columns=X_train.columns)

In [10]:
X_train_nn = preprocessor.fit_transform(X_train_nn)
X_val_nn = preprocessor.transform(X_val_nn)
X_test_nn = preprocessor.transform(X_test)

scaler = StandardScaler()
X_train_nn = scaler.fit_transform(X_train_nn)
X_val_nn = scaler.transform(X_val_nn)
X_test_nn = scaler.transform(X_test_nn)

In [11]:
y_train_nn = y_train_nn.values.reshape(-1, 1)
y_val_nn = y_val_nn.values.reshape(-1, 1)
y_test_nn = y_test.values.reshape(-1, 1)

In [12]:
y_train_nn = tf.cast(y_train_nn, dtype=tf.float32)
y_val_nn = tf.cast(y_val_nn, dtype=tf.float32)
y_test_nn = tf.cast(y_test_nn, dtype=tf.float32)

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_nn, y_train_nn))
train_dataset = train_dataset.shuffle(buffer_size=6000).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val_nn, y_val_nn))
val_dataset = val_dataset.batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices(X_test_nn)
test_dataset = test_dataset.batch(32)


In [14]:

def build_model(hp):
  n_hidden = hp.Int("n_hidden", min_value=1, max_value=20)
  n_neurons = hp.Int("n_neurons", min_value=16, max_value=1000)
  learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-1, sampling="log")
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten())
  for _ in range(n_hidden):
    model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
  model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

  model.compile(loss="binary_crossentropy",
                optimizer=optimizer,
                metrics=[tf.keras.metrics.F1Score(name='f1', average='micro', threshold=0.5)])
  return model

In [15]:

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_f1', patience = 5, mode='max', restore_best_weights = True)
callbacks = [early_stopping_cb]

random_search_tuner = kt.RandomSearch(build_model, objective=kt.Objective(name="val_f1", direction="max"), max_trials = 10,
                                      overwrite=True, seed=42)

random_search_tuner.search(train_dataset, epochs=50,
                           validation_data=val_dataset,
                           callbacks=callbacks)

Trial 10 Complete [00h 10m 04s]
val_f1: 0.7494108080863953

Best val_f1 So Far: 0.7755101919174194
Total elapsed time: 00h 27m 24s


In [16]:
random_search_tuner.get_best_hyperparameters()[0].values

{'n_hidden': 5, 'n_neurons': 979, 'lr': 0.002941480046354757}

In [17]:
best_model = random_search_tuner.get_best_models()[0]
y_new_pred = [1 if p > 0.5 else 0 for p in best_model.predict(X_test_nn)]
log_results('DNN', y_test, y_new_pred)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
confusion matrix for DNN:
 [[ 601  527]
 [ 209 1200]]
Precision Score: 0.69
Recall Score: 0.85
F1 Score: 0.77
ROC_AUC Score: 0.69


In [18]:
best_model.save('dnn.keras')

# Compare Results

In [ ]:
headers = ["Model", "Precision", "Recall", "F1", "ROC_AUC"]
print(tabulate(results, headers, tablefmt="grid"))

+---------------+-------------+----------+----------+-----------+
| Model         |   Precision |   Recall |       F1 |   ROC_AUC |
+===============+=============+==========+==========+===========+
| SGDClassifier |    0.602642 | 0.841732 | 0.702399 |  0.574235 |
+---------------+-------------+----------+----------+-----------+
| Decision Tree |    0.700379 | 0.786373 | 0.740889 |  0.68308  |
+---------------+-------------+----------+----------+-----------+
| Random Forest |    0.717351 | 0.830376 | 0.769737 |  0.710844 |
+---------------+-------------+----------+----------+-----------+
| SVM           |    0.652174 | 0.81973  | 0.726415 |  0.636816 |
+---------------+-------------+----------+----------+-----------+
| DNN           |    0.716635 | 0.804116 | 0.75786  |  0.703477 |
+---------------+-------------+----------+----------+-----------+
